## Imports and loading data

In [2]:
import tensorflow as tf
import numpy as np

In [4]:
import tensorflow_datasets as tfds
from tensorflow.keras.utils import to_categorical

(train_ds, train_labels), (test_ds, test_labels) = tfds.load(
    "tf_flowers",
    split=["train[:70%]", "train[:30%]"],
    batch_size=-1,
    as_supervised=True,  # Include labels
)

In [5]:
train_ds.shape

TensorShape([2569, 442, 1024, 3])

## Preprocessing data

In [6]:
size = (150, 150)

train_ds = tf.image.resize(train_ds, (150, 150))
test_ds = tf.image.resize(test_ds, (150, 150))

train_labels = to_categorical(train_labels, num_classes=5)
test_labels = to_categorical(test_labels, num_classes=5)

In [7]:
train_ds.shape

TensorShape([2569, 150, 150, 3])

## Loading VGG16 model

In [11]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

train_ds = preprocess_input(train_ds) 
test_ds = preprocess_input(test_ds)

base_model = VGG16(weights="imagenet", include_top=False, input_shape=train_ds[0].shape)
base_model.trainable = False


In [12]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

## Adding Layers

In [9]:
from tensorflow.keras import layers, models


flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(50, activation='relu')
dense_layer_2 = layers.Dense(20, activation='relu')
prediction_layer = layers.Dense(5, activation='softmax')



model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])


In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 50)                409650    
_________________________________________________________________
dense_1 (Dense)              (None, 20)                1020      
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 105       
Total params: 15,125,463
Trainable params: 410,775
Non-trainable params: 14,714,688
_________________________________________________________________


## Training model

In [12]:
from tensorflow.keras.callbacks import EarlyStopping

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)


es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)

model.fit(train_ds, train_labels, epochs=50, validation_split=0.2, batch_size=32, callbacks=[es])

Epoch 1/50
65/65 [==============================] - 6s 44ms/step - loss: 1.6855 - accuracy: 0.3810 - val_loss: 1.2690 - val_accuracy: 0.4572
Epoch 2/50
65/65 [==============================] - 2s 30ms/step - loss: 1.0561 - accuracy: 0.5742 - val_loss: 1.1122 - val_accuracy: 0.5409
Epoch 3/50
65/65 [==============================] - 2s 30ms/step - loss: 0.7746 - accuracy: 0.7100 - val_loss: 1.0297 - val_accuracy: 0.6187
Epoch 4/50
65/65 [==============================] - 2s 30ms/step - loss: 0.5840 - accuracy: 0.7956 - val_loss: 1.0904 - val_accuracy: 0.6537
Epoch 5/50
65/65 [==============================] - 2s 30ms/step - loss: 0.4690 - accuracy: 0.8316 - val_loss: 1.1269 - val_accuracy: 0.6615
Epoch 6/50
65/65 [==============================] - 2s 30ms/step - loss: 0.3728 - accuracy: 0.8745 - val_loss: 1.1244 - val_accuracy: 0.6712
Epoch 7/50
65/65 [==============================] - 2s 30ms/step - loss: 0.2928 - accuracy: 0.8925 - val_loss: 1.1643 - val_accuracy: 0.6712
Epoch 8/50
65

In [13]:
model.evaluate(test_ds, test_labels)

35/35 [==============================] - 1s 40ms/step - loss: 0.1252 - accuracy: 0.9619


[0.1252465844154358, 0.9618528485298157]

## Hand Made Model

In [14]:
from tensorflow.keras import Sequential, layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Rescaling


hand_made_model = Sequential()
hand_made_model.add(Rescaling(1./255, input_shape=(150,150,3)))

hand_made_model.add(layers.Conv2D(16, kernel_size=10, activation='relu'))
hand_made_model.add(layers.MaxPooling2D(3))

hand_made_model.add(layers.Conv2D(32, kernel_size=8, activation="relu"))
hand_made_model.add(layers.MaxPooling2D(2))

hand_made_model.add(layers.Conv2D(32, kernel_size=6, activation="relu"))
hand_made_model.add(layers.MaxPooling2D(2))

hand_made_model.add(layers.Flatten())
hand_made_model.add(layers.Dense(50, activation='relu'))
hand_made_model.add(layers.Dense(20, activation='relu'))
hand_made_model.add(layers.Dense(5, activation='softmax'))


hand_made_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)


es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)

hand_made_model.fit(train_ds, train_labels, epochs=50, validation_split=0.2, batch_size=32, callbacks=[es])

Epoch 1/50
65/65 [==============================] - 4s 18ms/step - loss: 1.5190 - accuracy: 0.2895 - val_loss: 1.4304 - val_accuracy: 0.3152
Epoch 2/50
65/65 [==============================] - 1s 12ms/step - loss: 1.3129 - accuracy: 0.4180 - val_loss: 1.2735 - val_accuracy: 0.4825
Epoch 3/50
65/65 [==============================] - 1s 13ms/step - loss: 1.2217 - accuracy: 0.4706 - val_loss: 1.2042 - val_accuracy: 0.4844
Epoch 4/50
65/65 [==============================] - 1s 13ms/step - loss: 1.1505 - accuracy: 0.5187 - val_loss: 1.1823 - val_accuracy: 0.5058
Epoch 5/50
65/65 [==============================] - 1s 13ms/step - loss: 1.0741 - accuracy: 0.5489 - val_loss: 1.1598 - val_accuracy: 0.5117
Epoch 6/50
65/65 [==============================] - 1s 13ms/step - loss: 1.0077 - accuracy: 0.5961 - val_loss: 1.3546 - val_accuracy: 0.5117
Epoch 7/50
65/65 [==============================] - 1s 12ms/step - loss: 0.9820 - accuracy: 0.6083 - val_loss: 1.1733 - val_accuracy: 0.5292
Epoch 8/50
65

In [15]:
hand_made_model.evaluate(test_ds, test_labels)

35/35 [==============================] - 0s 9ms/step - loss: 0.4268 - accuracy: 0.8320


[0.42677685618400574, 0.8319709300994873]